In [1]:
import math
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
customers_df = pd.read_csv("../data/olist_customers_dataset.csv")
geoloc_df = pd.read_csv("../data/olist_geolocation_dataset.csv")
order_items_df = pd.read_csv("../data/olist_order_items_dataset.csv")
order_payments_df = pd.read_csv("../data/olist_order_payments_dataset.csv")
order_reviews_df = pd.read_csv("../data/olist_order_reviews_dataset.csv")
order_df = pd.read_csv("../data/olist_orders_dataset.csv")
products_df = pd.read_csv("../data/olist_products_dataset.csv")
sellers_df = pd.read_csv("../data/olist_sellers_dataset.csv")
product_category_name_translation_df = pd.read_csv("../data/product_category_name_translation.csv")

In [3]:
order_df.dropna(inplace=True)

In [4]:
def infer_datetime(s):
    return datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

def get_lat_lng(uid:str, uid_type:str):
    
    if uid_type == "seller":
        lat = sellers_df[sellers_df.seller_id == uid]['s_lat'].values[0]
        lng = sellers_df[sellers_df.seller_id == uid]['s_lng'].values[0]
    
    elif uid_type == "customer":
        lat = customers_df[customers_df.customer_id == uid]['c_lat'].values[0]
        lng = customers_df[customers_df.customer_id == uid]['c_lng'].values[0]
    return lat,lng

def get_lat_lon_dist(lat1,lon1,lat2,lon2):
    try:
        return round(math.acos((math.sin(math.radians(lat1)) * math.sin(math.radians(lat2))) + 
                (math.cos(math.radians(lat1)) * math.cos(math.radians(lat2))) * 
                (math.cos(math.radians(lon2) - math.radians(lon1)))) * 6371, 3)
    except ValueError:
        print(lat1,lon1,lat2,lon2)
        return 0

def get_customer_seller_dist(customer_id, seller_id):
    lat1, lng1 = get_lat_lng(customer_id, 'customer')
    seller_dists = []
    for s in seller_id:
        lat2, lng2 = get_lat_lng(s, 'seller')
        seller_dists.append(get_lat_lon_dist(lat1, lng1, lat2, lng2))
    return max(seller_dists)

def get_business_days(start_date, end_date):
    return sum([date.isoweekday() <=5 for date in pd.date_range(infer_datetime(start_date), infer_datetime(end_date))])

In [5]:
products_df['product_volumn'] = products_df.apply(lambda r: r.product_length_cm*r.product_height_cm*r.product_width_cm, axis=1)

order_items_df = order_items_df.merge(products_df, on="product_id")

grouped_order_items_df = order_items_df.groupby(['order_id']).agg({'product_id': ['count', 'nunique'], 
                                          'seller_id': ['count', 'nunique', 'unique'],
                                          'price':'sum', 'freight_value':'sum',
                                          'product_volumn':'sum', 'product_weight_g':'sum'}
                                        ).reset_index()

grouped_order_items_df.columns = ['_'.join([col1,col2]) if col2 else col1 for col1,col2 in grouped_order_items_df.columns.values]

grouped_order_items_df.reset_index(drop=True, inplace=True)
# grouped_order_items_df

In [6]:
full_df = pd.merge(left = order_df, right=grouped_order_items_df, on='order_id', how='left')

In [7]:
full_df['order_approved_after(s)'] = full_df.apply(lambda r: (infer_datetime(r.order_approved_at) - infer_datetime(r.order_purchase_timestamp)).total_seconds(), axis=1)

In [8]:
full_df['order_delivered_carrier_after_approval(s)'] = full_df.apply(lambda r: (infer_datetime(r.order_delivered_carrier_date) - infer_datetime(r.order_approved_at)).total_seconds(), axis=1)

In [9]:
grouped_order_payments_df = order_payments_df.groupby(['order_id']).agg({'payment_sequential': 'max', 
                                          'payment_type': lambda x: pd.Series(list(x)).value_counts().to_dict(),
                                          'payment_installments':'sum', 
                                          'payment_value':'sum'}
                                        ).reset_index()

In [10]:
grouped_order_payments_df['credit_card_payments'] = grouped_order_payments_df.payment_type.apply(lambda x: x.get('credit_card',0))
grouped_order_payments_df['boleto_payments'] = grouped_order_payments_df.payment_type.apply(lambda x: x.get('boleto',0))
grouped_order_payments_df['voucher_payments'] = grouped_order_payments_df.payment_type.apply(lambda x: x.get('voucher',0))
grouped_order_payments_df['debit_card_payments'] = grouped_order_payments_df.payment_type.apply(lambda x: x.get('debit_card',0))
grouped_order_payments_df['not_defined_payments'] = grouped_order_payments_df.payment_type.apply(lambda x: x.get('not_defined',0))

In [11]:
full_df = pd.merge(left = full_df, right=grouped_order_payments_df, on='order_id', how='left')

In [12]:
geoloc_df['geo_id'] = geoloc_df.geolocation_zip_code_prefix.astype("str")+geoloc_df.geolocation_city + geoloc_df.geolocation_state
customers_df['geo_id'] = customers_df['customer_zip_code_prefix'].astype("str")+customers_df.customer_city+customers_df.customer_state
sellers_df['geo_id'] = sellers_df['seller_zip_code_prefix'].astype("str")+sellers_df.seller_city+sellers_df.seller_state

geoloc_df_grouped = geoloc_df.groupby("geo_id").agg({"geolocation_lat":"mean",
                                "geolocation_lng":"mean"})
geoloc_df_grouped.reset_index(inplace=True)

customers_df = customers_df.merge(geoloc_df_grouped, on='geo_id', how='left').rename({'geolocation_lat': 'c_lat', 'geolocation_lng': 'c_lng'}, axis=1)

sellers_df = sellers_df.merge(geoloc_df_grouped, on='geo_id', how='left').rename({'geolocation_lat': 's_lat', 'geolocation_lng': 's_lng'}, axis=1)

In [13]:
full_df['seller_lat_lon'] = full_df.progress_apply(lambda r: np.mean([get_lat_lng(sid, 'seller') for sid in r.seller_id_unique],axis=0), axis=1)

full_df['cust_lat_lon'] = full_df.progress_apply(lambda r: get_lat_lng(r.customer_id, 'customer'), axis=1)

full_df['seller_cust_dist'] = full_df.progress_apply(lambda r: get_lat_lon_dist(r.cust_lat_lon[0],r.cust_lat_lon[1],
                                                                                r.seller_lat_lon[0],r.seller_lat_lon[1]), axis=1)

  0%|          | 0/96461 [00:00<?, ?it/s]

  0%|          | 0/96461 [00:00<?, ?it/s]

  0%|          | 0/96461 [00:00<?, ?it/s]

-24.95918428429227 -53.462644000056976 -24.95918428429227 -53.462644000056976


In [15]:
full_df['late_delivery'] = order_df.apply(lambda r: infer_datetime(r.order_delivered_customer_date) > infer_datetime(r.order_estimated_delivery_date), axis=1)
full_df['late_delivery'].value_counts(normalize=True)

late_delivery
False    0.918823
True     0.081177
Name: proportion, dtype: float64

In [16]:
full_df.dropna(inplace=True)
full_df.reset_index(drop=True, inplace=True)

In [17]:
full_df['late_delivery'].value_counts(normalize=True)

late_delivery
False    0.918718
True     0.081282
Name: proportion, dtype: float64

In [18]:
full_df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'product_id_count', 'product_id_nunique', 'seller_id_count',
       'seller_id_nunique', 'seller_id_unique', 'price_sum',
       'freight_value_sum', 'product_volumn_sum', 'product_weight_g_sum',
       'order_approved_after(s)', 'order_delivered_carrier_after_approval(s)',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'credit_card_payments', 'boleto_payments',
       'voucher_payments', 'debit_card_payments', 'not_defined_payments',
       'seller_lat_lon', 'cust_lat_lon', 'seller_cust_dist', 'late_delivery'],
      dtype='object')

In [19]:
full_df['deliverd_to_carrier_after(business_days)'] = full_df.apply(lambda r: get_business_days(start_date = r.order_purchase_timestamp,
                                                                                                end_date = r.order_delivered_carrier_date), axis=1)

In [20]:
full_df['deliverd_to_carrier_after(business_days)'].value_counts()

deliverd_to_carrier_after(business_days)
2     27286
1     25339
3     14820
4      8074
5      4615
0      3266
6      2288
7      1381
8       897
9       621
10      511
11      374
12      261
13      208
14      151
15      119
16       99
17       77
18       70
19       41
20       38
21       26
22       21
29       20
24       20
23       20
25       18
30       16
27       12
28       10
31       10
26        9
34        7
33        4
36        4
32        3
40        2
75        1
39        1
78        1
90        1
37        1
47        1
45        1
44        1
Name: count, dtype: int64

In [21]:
full_df['deliverd_to_customer_after(business_days)'] = full_df.apply(lambda r: get_business_days(start_date = r.order_delivered_carrier_date,
                                                                                                 end_date = r.order_delivered_customer_date), axis=1)

In [22]:
full_df['deliverd_to_carrier_after(business_days)'].value_counts()

deliverd_to_carrier_after(business_days)
2     27286
1     25339
3     14820
4      8074
5      4615
0      3266
6      2288
7      1381
8       897
9       621
10      511
11      374
12      261
13      208
14      151
15      119
16       99
17       77
18       70
19       41
20       38
21       26
22       21
29       20
24       20
23       20
25       18
30       16
27       12
28       10
31       10
26        9
34        7
33        4
36        4
32        3
40        2
75        1
39        1
78        1
90        1
37        1
47        1
45        1
44        1
Name: count, dtype: int64

In [95]:
# order_payments_df[order_payments_df.order_id == "fa65dad1b0e818e3ccc5cb0e39231352"]

In [24]:
full_df.to_csv("olist_merged.csv", index=False)

### Model 1 : Predict the number of business days to deliver the order to carrier

In [153]:
model1_data = full_df.loc[:, ['order_id','product_id_count','product_id_nunique','seller_id_count',
                              'seller_id_nunique','price_sum','freight_value_sum','product_volumn_sum', 
                              'product_weight_g_sum', 'order_approved_after(s)', 'payment_type', 'payment_installments', 
                              'seller_cust_dist','deliverd_to_carrier_after(business_days)']]

In [154]:
model1_data

,order_id,product_id_count,product_id_nunique,seller_id_count,seller_id_nunique,price_sum,freight_value_sum,product_volumn_sum,product_weight_g_sum,order_approved_after(s),payment_type,payment_installments,seller_cust_dist,deliverd_to_carrier_after(business_days)
0,e481f51cbdc54678b7cc49136f2d6af7,1,1,1,1,29.99,8.72,1976.0,500.0,642.0,3.0,3.0,18.581,3
1,47770eb9100c2d0c44946d9cf07ec65d,1,1,1,1,159.90,19.22,9576.0,420.0,994.0,1.0,3.0,514.448,1
2,949d5b44dbf5de918fe9c16f97b45f8a,1,1,1,1,45.00,27.20,6000.0,450.0,1073.0,1.0,1.0,1822.109,2
3,ad21c59c0840e6cb83a9ceb5573f8159,1,1,1,1,19.90,8.72,11475.0,250.0,3710.0,1.0,1.0,29.678,1
4,a4591c265e18cb1dcee52889e2d8acc3,1,1,1,1,147.90,27.36,42250.0,7150.0,788.0,1.0,6.0,411.394,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90741,9c5dedf39a927c1b2549525ed64a053c,1,1,1,1,72.00,13.08,5148.0,1175.0,0.0,1.0,3.0,69.777,2
90742,63943bddc261676b46f01ca7ac2f7bd8,1,1,1,1,174.90,20.10,16000.0,4950.0,699.0,1.0,3.0,474.100,2
90743,83c1379a015df1e13d02aae0204711ab,1,1,1,1,205.99,65.02,63360.0,13300.0,1053.0,1.0,5.0,967.794,1
90744,11c177c8e97725db2631073c19f07b62,2,1,2,1,359.98,81.18,16000.0,13100.0,474.0,1.0,4.0,370.346,4


In [150]:
from sklearn.preprocessing import StandardScaler

In [159]:
num_cols = ['price_sum','freight_value_sum', 'product_volumn_sum', 'product_weight_g_sum', 'order_approved_after(s)',
            'seller_cust_dist']

In [160]:
scalar = StandardScaler()
model1_data.loc[:,(num_cols)] = scalar.fit_transform(model1_data.loc[:,(num_cols)])

In [161]:
model1_data

,order_id,product_id_count,product_id_nunique,seller_id_count,seller_id_nunique,price_sum,freight_value_sum,product_volumn_sum,product_weight_g_sum,order_approved_after(s),payment_type,payment_installments,seller_cust_dist,deliverd_to_carrier_after(business_days)
0,e481f51cbdc54678b7cc49136f2d6af7,1,1,1,1,-0.508937,-0.649233,-0.508171,-0.392535,-0.490818,3.0,3.0,-0.980211,3
1,47770eb9100c2d0c44946d9cf07ec65d,1,1,1,1,0.113900,-0.160615,-0.252900,-0.409606,-0.486070,1.0,3.0,-0.144942,1
2,949d5b44dbf5de918fe9c16f97b45f8a,1,1,1,1,-0.436973,0.210735,-0.373012,-0.403204,-0.485005,1.0,1.0,2.057761,2
3,ad21c59c0840e6cb83a9ceb5573f8159,1,1,1,1,-0.557312,-0.649233,-0.189115,-0.445881,-0.449438,1.0,1.0,-0.961518,1
4,a4591c265e18cb1dcee52889e2d8acc3,1,1,1,1,0.056368,0.218181,0.844567,1.026474,-0.488849,1.0,6.0,-0.318533,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90741,9c5dedf39a927c1b2549525ed64a053c,1,1,1,1,-0.307525,-0.446340,-0.401629,-0.248500,-0.499477,1.0,3.0,-0.893973,2
90742,63943bddc261676b46f01ca7ac2f7bd8,1,1,1,1,0.185816,-0.119664,-0.037128,0.557027,-0.490049,1.0,3.0,-0.212907,2
90743,83c1379a015df1e13d02aae0204711ab,1,1,1,1,0.334873,1.970692,1.553617,2.338791,-0.485275,1.0,5.0,0.618701,1
90744,11c177c8e97725db2631073c19f07b62,2,1,2,1,1.073159,2.722698,-0.037128,2.296114,-0.493084,1.0,4.0,-0.387676,4


In [162]:
X = model1_data.drop(['order_id', 'deliverd_to_carrier_after(business_days)'], axis=1)
y = model1_data.loc[:,'deliverd_to_carrier_after(business_days)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,  random_state=314)

In [227]:
y_cut = pd.cut(y,bins=[0,1,3,6,11,9999],
               include_lowest=True,
               right=False,
               labels=['same_day','1_2_days','3_5_days','6_10_days','more_than_10_days']
              )

X_train, X_test, y_train, y_test = train_test_split(X, y_cut, test_size = 0.20,  random_state=314)

In [232]:
rf_classifier = RandomForestClassifier(max_depth=10)
rf_classifier.fit(X_train, y_train)
y_train_pred = rf_classifier.predict(X_train)
y_test_pred = rf_classifier.predict(X_test)

In [233]:
print(f"Train F1 Score {f1_score(y_train, y_train_pred, average='micro')}")
print(f"Test F1 Score {f1_score(y_test, y_test_pred, average='micro')}")

Train F1 Score 0.6346768416992672
Test F1 Score 0.6155922865013774


In [163]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error

In [164]:
def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    return model, y_train_pred, y_test_pred

In [165]:
lr_regressor = LinearRegression()
lr_regressor.fit(X_train, y_train)
y_train_pred = lr_regressor.predict(X_train)
y_test_pred = lr_regressor.predict(X_test)

rounder = lambda t: np.round(t)
y_train_pred = rounder(y_train_pred)
y_test_pred = rounder(y_test_pred)

In [166]:
print(f"Train F1 Score {mean_squared_error(y_train, y_train_pred)}")
print(f"Test F1 Score {mean_squared_error(y_test, y_test_pred)}")

Train F1 Score 6.284519808253898
Test F1 Score 6.385289256198347


In [167]:
rf_regressor, y_train_pred, y_test_pred = fit_model(RandomForestRegressor(), X_train, y_train)

In [168]:
print(f"Train F1 Score {mean_squared_error(y_train, y_train_pred)}")
print(f"Test F1 Score {mean_squared_error(y_test, y_test_pred)}")

Train F1 Score 0.8017706099509615
Test F1 Score 5.746675173553719


In [169]:
from xgboost import XGBRegressor

In [170]:
xgb_regressor, y_train_pred, y_test_pred = fit_model(XGBRegressor(), X_train, y_train)
print(f"Train F1 Score {mean_squared_error(y_train, y_train_pred)}")
print(f"Test F1 Score {mean_squared_error(y_test, y_test_pred)}")

Train F1 Score 4.034952653169647
Test F1 Score 5.695081868431639


In [171]:
rounder = lambda t: np.round(t)
y_train_pred = rounder(y_train_pred)
y_test_pred = rounder(y_test_pred)
print(f"Train F1 Score {mean_squared_error(y_train, y_train_pred)}")
print(f"Test F1 Score {mean_squared_error(y_test, y_test_pred)}")

Train F1 Score 4.11128712325748
Test F1 Score 5.780716253443527


In [21]:
delayed_delivery_data = full_df.loc[:, ['order_id','product_id_count','product_id_nunique','seller_id_count',
                                        'seller_id_nunique','price_sum','freight_value_sum','product_volumn_sum', 
                                        'product_weight_g_sum', 'order_approved_after(s)', 
                                        'order_delivered_carrier_after_approval(s)',  'payment_type', 
                                        'payment_installments', 'payment_value','seller_cust_dist','late_delivery']]

In [94]:
delayed_delivery_data

,order_id,product_id_count,product_id_nunique,seller_id_count,seller_id_nunique,price_sum,freight_value_sum,product_volumn_sum,product_weight_g_sum,order_approved_after(s),order_delivered_carrier_after_approval(s),payment_type,payment_installments,payment_value,seller_cust_dist,late_delivery
0,e481f51cbdc54678b7cc49136f2d6af7,1,1,1,1,29.99,8.72,1976.0,500.0,642.0,204465.0,3.0,3.0,38.71,18.581,False
1,47770eb9100c2d0c44946d9cf07ec65d,1,1,1,1,159.90,19.22,9576.0,420.0,994.0,17677.0,1.0,3.0,179.12,514.448,False
2,949d5b44dbf5de918fe9c16f97b45f8a,1,1,1,1,45.00,27.20,6000.0,450.0,1073.0,323640.0,1.0,1.0,72.20,1822.109,False
3,ad21c59c0840e6cb83a9ceb5573f8159,1,1,1,1,19.90,8.72,11475.0,250.0,3710.0,77165.0,1.0,1.0,28.62,29.678,False
4,a4591c265e18cb1dcee52889e2d8acc3,1,1,1,1,147.90,27.36,42250.0,7150.0,788.0,146871.0,1.0,6.0,175.26,411.394,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90741,9c5dedf39a927c1b2549525ed64a053c,1,1,1,1,72.00,13.08,5148.0,1175.0,0.0,91438.0,1.0,3.0,85.08,69.777,False
90742,63943bddc261676b46f01ca7ac2f7bd8,1,1,1,1,174.90,20.10,16000.0,4950.0,699.0,123125.0,1.0,3.0,195.00,474.100,False
90743,83c1379a015df1e13d02aae0204711ab,1,1,1,1,205.99,65.02,63360.0,13300.0,1053.0,107290.0,1.0,5.0,271.01,967.794,False
90744,11c177c8e97725db2631073c19f07b62,2,1,2,1,359.98,81.18,16000.0,13100.0,474.0,323922.0,1.0,4.0,441.16,370.346,False


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [38]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
y_train_pred = rf_classifier.predict(X_train)
y_test_pred = rf_classifier.predict(X_test)

In [39]:
print(f"Train F1 Score {f1_score(y_train, y_train_pred)}")
print(f"Test F1 Score {f1_score(y_test, y_test_pred)}")

Train F1 Score 0.999744092809008
Test F1 Score 0.0


In [40]:
rf_classifier = RandomForestClassifier(max_depth=20, class_weight='balanced')
rf_classifier.fit(X_train, y_train)
y_train_pred = rf_classifier.predict(X_train)
y_test_pred = rf_classifier.predict(X_test)

In [41]:
print(f"Train F1 Score {f1_score(y_train, y_train_pred)}")
print(f"Test F1 Score {f1_score(y_test, y_test_pred)}")

Train F1 Score 0.9046403099410055
Test F1 Score 0.016568047337278107


In [91]:
full_df.customer_id.value_counts()

customer_id
9ef432eb6251297304e76186b10a928d    1
4ff9d132fe2c07793f39e324e86f0383    1
a51921054a93c2d99f085ebee1817c81    1
82eb0f700864871ddc51572601014ba4    1
b0d798d76605a085ed8b3a196da24b24    1
                                   ..
41719c34900034230304c9266e5b6713    1
2d0ef7c75a0403680d304c62ba3b2721    1
f86ef9a9bd878f3c22617b38ea06d3ff    1
dbad14e1cbdb1f106ac9b2b3f4b89d15    1
edb027a75a1449115f6b43211ae02a24    1
Name: count, Length: 90746, dtype: int64

In [18]:
# full_df['same_day_delivery'] = full_df.apply(lambda r: datetime.strftime(infer_datetime(r.order_purchase_timestamp), format="%Y-%m-%d") ==
#                                              datetime.strftime(infer_datetime(r.order_delivered_customer_date), format="%Y-%m-%d"), axis=1)

In [18]:
test_sample = full_df.sample(50)

In [19]:
test_sample['seller_cust_dist'] = test_sample.progress_apply(lambda r: get_customer_seller_dist(r.customer_id, r.seller_id_unique), axis=1)

  0%|          | 0/50 [00:00<?, ?it/s]

In [20]:
test_sample

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,target,product_id_count,...,freight_value_sum,product_volumn_sum,product_weight_g_sum,order_approved_after(s),order_delivered_carrier_after_approval(s),payment_sequential,payment_type,payment_installments,payment_value,seller_cust_dist
83491,12fd4ba303cb9951f45b3ce9b6ba764b,8b4639324f4de248b5196a893aaad0eb,delivered,2018-03-18 21:13:46,2018-03-18 21:30:28,2018-03-19 22:46:39,2018-04-10 22:14:46,2018-04-13 00:00:00,False,1,...,22.85,3024.0,247.0,1002.0,90971.0,1.0,1.0,3.0,58.75,2218.914
51970,86f353da4086de44a6acd7faf109e47d,dcdd2033f19ef7bcacdf2e683c8f2e65,delivered,2017-07-13 22:29:41,2017-07-13 22:45:07,2017-07-21 18:57:19,2017-08-01 18:37:43,2017-08-10 00:00:00,False,1,...,53.48,87696.0,16800.0,926.0,677532.0,7.0,7.0,9.0,263.47,324.296
71396,293fe99febcd4990b7ab31c5cd2abfd5,407d685edcbff0e025864e271eb2e5b7,delivered,2017-08-20 16:22:25,2017-08-20 16:35:13,2017-08-22 10:33:43,2017-08-24 18:43:51,2017-09-05 00:00:00,False,1,...,7.78,4500.0,200.0,768.0,151110.0,1.0,1.0,4.0,42.77,34.106
45322,03e6de4517499ee6aaae7d9004799504,650b8d581bd67d61888f8ea779e34e7c,delivered,2018-05-16 18:39:40,2018-05-16 18:57:28,2018-05-17 13:45:00,2018-05-29 14:36:55,2018-06-26 00:00:00,False,1,...,23.98,4352.0,767.0,1068.0,67652.0,1.0,1.0,2.0,173.88,729.701
43312,a3a0cfe6661449e2d725b5b1ff658d84,2e57ca0d2c4b939fd528fe677acca228,delivered,2018-07-10 11:52:24,2018-07-10 12:05:32,2018-07-11 08:40:00,2018-07-20 19:55:33,2018-08-01 00:00:00,False,1,...,27.22,6750.0,1250.0,788.0,74068.0,1.0,1.0,2.0,92.12,1687.832
94616,9286a7cc933d6e2099098e4ae057e5dc,ed8097f13c2401a34f2d78cf43260846,delivered,2018-03-27 03:01:08,2018-03-28 02:50:14,2018-03-28 20:03:12,2018-04-02 20:05:18,2018-04-13 00:00:00,False,1,...,12.79,816.0,200.0,85746.0,61978.0,1.0,1.0,1.0,34.69,28.523
1031,241a57ba96decfbe134f5c84f585f57f,b8426f2cfe27d43ae8077d218a3622ad,delivered,2017-11-10 17:21:48,2017-11-10 17:35:33,2017-11-14 16:47:09,2017-11-22 00:22:48,2017-12-01 00:00:00,False,1,...,15.79,3328.0,275.0,825.0,342696.0,1.0,1.0,1.0,46.79,882.187
84659,b997ce40f469fbcd2e163e6a2df04de8,e193e498ca7e4b73bf1d55f365b6f64b,delivered,2017-03-25 14:30:46,2017-03-25 14:45:11,2017-03-28 11:42:59,2017-04-03 16:53:51,2017-04-19 00:00:00,False,1,...,14.11,1980.0,150.0,865.0,248268.0,1.0,1.0,3.0,32.10,350.575
8349,d3f58dcb7705f30c2257940517740287,cbcfae7b60710216bbea3d587a83af2e,delivered,2017-05-02 03:48:26,2017-05-02 04:02:33,2017-05-02 14:48:05,2017-05-05 16:43:38,2017-05-29 00:00:00,False,1,...,19.45,8000.0,2200.0,847.0,38732.0,1.0,1.0,5.0,208.35,912.222
18963,0b3f84ce012f341897587c6e4e73f68e,bd2e650f73a529c3dd307b6eec01393e,delivered,2018-06-30 21:48:45,2018-07-01 21:50:01,2018-07-03 11:15:00,2018-07-12 15:28:49,2018-08-01 00:00:00,False,1,...,60.20,58682.0,10050.0,86476.0,134699.0,1.0,1.0,5.0,198.20,311.062


In [44]:
test_sample[test_sample['seller_cust_dist'] == test_sample['seller_cust_dist'].max()].iloc[0]

order_id                                       c1b86a370f491b37cc3adb8ef0a21fe5
customer_id                                    5c5689c7522389a8ff3e28ead3a4cd55
order_status                                                          delivered
order_purchase_timestamp                                    2017-03-21 10:09:06
order_approved_at                                           2017-03-26 23:32:51
order_delivered_carrier_date                                2017-03-30 16:32:07
order_delivered_customer_date                               2017-04-20 08:05:23
order_estimated_delivery_date                               2017-04-28 00:00:00
target                                                                    False
product_id_count                                                              1
product_id_nunique                                                            1
seller_id_count                                                               1
seller_id_nunique                       

In [45]:
test_sample[test_sample['seller_cust_dist'] == test_sample['seller_cust_dist'].min()].iloc[0]

order_id                                       d55ece0f3afa9b57b5a78038efddbbbc
customer_id                                    f027649d8f2a7b182522d5da3cd4f010
order_status                                                          delivered
order_purchase_timestamp                                    2017-05-22 17:20:34
order_approved_at                                           2017-05-24 02:50:18
order_delivered_carrier_date                                2017-05-24 09:44:28
order_delivered_customer_date                               2017-05-25 10:40:49
order_estimated_delivery_date                               2017-06-06 00:00:00
target                                                                    False
product_id_count                                                              1
product_id_nunique                                                            1
seller_id_count                                                               1
seller_id_nunique                       

In [134]:
customers_df[customers_df.customer_id == "948b29e24216a05fea13a18d8db45ea5"]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geo_id,c_lat,c_lng
84222,948b29e24216a05fea13a18d8db45ea5,6740f8899f3c70b5b08b2e0bad37e567,83252,ilha dos valadares,PR,83252ilha dos valadaresPR,42.184003,-8.723762


In [135]:
sellers_df[sellers_df.seller_id == "0417b067eeab773d2f7061a726dc477f"]

,seller_id,seller_zip_code_prefix,seller_city,seller_state,geo_id,s_lat,s_lng
2658,0417b067eeab773d2f7061a726dc477f,89288,sao bento do sul,SC,89288sao bento do sulSC,-26.232399,-49.411493


In [136]:
get_lat_lng('948b29e24216a05fea13a18d8db45ea5', 'customer')

(42.18400274298598, -8.723762147513938)

In [137]:
get_lat_lng('0417b067eeab773d2f7061a726dc477f', 'seller')

(-26.23239898377447, -49.41149336148303)

In [138]:
get_lat_lon_dist(42.18400274298598, -8.723762147513938, -26.23239898377447, -49.41149336148303)

8677.912

In [132]:
full_df[full_df.customer_id == "5c5689c7522389a8ff3e28ead3a4cd55"].iloc[0]

order_id                                              c1b86a370f491b37cc3adb8ef0a21fe5
customer_id                                           5c5689c7522389a8ff3e28ead3a4cd55
order_status                                                                 delivered
order_purchase_timestamp                                           2017-03-21 10:09:06
order_approved_at                                                  2017-03-26 23:32:51
order_delivered_carrier_date                                       2017-03-30 16:32:07
order_delivered_customer_date                                      2017-04-20 08:05:23
order_estimated_delivery_date                                      2017-04-28 00:00:00
target                                                                           False
product_id_count                                                                     1
product_id_nunique                                                                   1
seller_id_count                            

In [133]:
full_df[full_df['seller_cust_dist'] == full_df['seller_cust_dist'].max()].iloc[0]

order_id                                             8ad3f1d0f96992e43566c4c82c9f6c58
customer_id                                          948b29e24216a05fea13a18d8db45ea5
order_status                                                                delivered
order_purchase_timestamp                                          2018-07-17 21:25:29
order_approved_at                                                 2018-07-17 21:35:17
order_delivered_carrier_date                                      2018-07-18 13:08:00
order_delivered_customer_date                                     2018-08-14 04:04:40
order_estimated_delivery_date                                     2018-08-03 00:00:00
target                                                                           True
product_id_count                                                                    1
product_id_nunique                                                                  1
seller_id_count                                       